In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import random

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [5]:
X_train = (X_train/255).reshape(-1,784)
X_test = (X_test/255).reshape(-1,784)
#X_train.shape
#-1:차원을 맞춰서 (,)해주면, 전체 개수에 맞춰서 나머지 차원을 보고 계산해줌(60000,784)와 같은 작업

In [6]:
#생성자 만들기 - 생성자망
from keras.layers.advanced_activations import LeakyReLU

g = Sequential()
g.add(Dense(units=256,input_dim=100,activation=LeakyReLU(0.2)))
g.add(Dense(units=512, activation=LeakyReLU(0.2)))
g.add(Dense(units=1024, activation=LeakyReLU(0.2)))

#units:이미지 데이터 2의n승(숫자 크게-256부터 시작)
#noise(random bector)로 가짜데이터 만듬 -> input_dim: 벡터의 feature수(차원수)
#0.2 : -1이 들어왔을 때 -1*0.2=-0.2만큼 함수값을 내림 

#출력층
g.add(Dense(units=784, activation='sigmoid'))
#units=내보내고 싶은 데이터 수
g.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
#판별자 망 -초기:분류기
d = Sequential()
d.add(Dense(units=1024,input_dim=784,activation=LeakyReLU(0.2)))
#input_dim=784 생성자가 출력층에서 784개의 데이터를 내보냄 
#생성자와 반대로 unit개수를 줄여가며 층을 쌓음
d.add(Dense(units=512,activation=LeakyReLU(0.2)))
d.add(Dense(units=512,activation=LeakyReLU(0.2)))
d.add(Dense(units=1,activation='sigmoid')) #출력증
#판별자에서 출력값 : 분류값(0,1)-> binary class '가짜다아니다'

d.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
d.trainable = False #훈련가능하지 않은 상태로 바꿔줌-> 학습X ->가중치갱신X
#판별자는 자신의 가중치를 가지고 있고 가짜데이터가 갖고있는 가중치를 계속 바꿔주지 않음(가짜데이터의 학습을 다시 안함-그저 판별만함)


In [ ]:
#노드가 여러개 있을때 점수들을 관리해줄때 -> softmax
#binary이므로 sigmoid

In [ ]:
#생성자와 판별자 층 쌓을 때 units의 수는 같은 값으로 서로 반대되도록 순서 지정하면 되는건가요?  꼭그런건아님
#아니면 생성자와 판별자의 unit의 숫자가 서로 반대되지 않더라도 생성자는 늘리고 판별자는 줄이기만 하면 되는건가요? 대체로 그렇다.

In [8]:
#생성자와 판별자를 연결한 신경망
from keras.models import Model
from keras.layers import Input #전체 간 구조에 해당하는 인풋만듬

#input층
gan_input = Input(shape=(100, ))
#만들고자하는 재료의 모양

#생성자에게 보내줌
x = g(inputs = gan_input)
#생성자가 받을 input의 모양

#중간결과물 판별자가 받음
gan_output = d(x)
#이를 받아서 최종적으로 내보냄 


#모델 쌓기
model = Model(gan_input, gan_output)#(입력,출력)
#Model : 알아서 층을 쌓아 모델완성해줌

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#판별자는 0or1 출력

In [9]:
#학습
EPOCHS = 10
BATCH_SIZE = 128

#60000개 데이터개수를 배치사이즈로 나눔
num_of_batches = int(X_train.shape[0] / BATCH_SIZE)

for e in range(EPOCHS):#반복한것을 에폭개수만큼 봄
    for n in range(num_of_batches): # 배치사이즈만큼 반복
        #4.판별자 학습#
        #가짜데이터 임의로 (재료) 생성
        noise = np.random.normal(0,1,size=(BATCH_SIZE,100))#0~1사이 숫자를 size개 생성
        ##배치사이즈 만큼의 실수값 (100차원)
        #가짜데이터 생성
        fake_data = g.predict(noise)
        #진짜데이터 생성
        real_data = X_train[np.random.randint(0, X_train.shape[0],size=BATCH_SIZE)]  #60000, 784-> 배치사이즈 만큼 랜덤하게 가져옴(정수값)
        
        #가짜,진짜데이터 합침(절반씩붙임)
        X = np.concatenate((real_data, fake_data))
        y = np.zeros(2 * BATCH_SIZE) #0으로 채워진 배열
        y[:BATCH_SIZE]=0.9 #절반은 0 이아닌 1이어야함(smooding해서 딱 1이아닌 0)
        
        #학습이 가능한 상태로 돌려줌
        d.trainable = True
        d_loss = d.train_on_batch(X,y) #간에서는 fit대신 씀->학습 (손실함수값 저장되어있음)
        
        #5.생성자 학습# - 간 전체를 학습시키는 것
        d.trainable = False #다시 판별자가 학습 못하게(판별만함 - 가중치 갱신X)
        #전체의 원재료 : noise로 가짜데이터 생성해서 판별자에게 넘기는 데이터 생성
        noise2 = np.random.normal(0,1,size=(BATCH_SIZE,100))
        y2 = np.ones(BATCH_SIZE)#0대신에 1로 채워진 배열 만들어줌
        g_loss = model.train_on_batch(noise2,y2) #학습
        
        print('EPOCH=',e+1, 'BATCH=',n+1, 'G_LOSS=',g_loss, 'D_LOSS=',d_loss)

EPOCH= 1 BATCH= 1 G_LOSS= [6.25101900100708, 0.0] D_LOSS= [0.6248681545257568, 0.5]
EPOCH= 1 BATCH= 2 G_LOSS= [3.66963791847229, 0.0] D_LOSS= [0.41628673672676086, 0.5]
EPOCH= 1 BATCH= 3 G_LOSS= [4.668774127960205, 0.0] D_LOSS= [0.20419344305992126, 0.5]
EPOCH= 1 BATCH= 4 G_LOSS= [7.823421478271484, 0.0] D_LOSS= [0.24220320582389832, 0.5]
EPOCH= 1 BATCH= 5 G_LOSS= [9.416372299194336, 0.0] D_LOSS= [0.20090064406394958, 0.5]
EPOCH= 1 BATCH= 6 G_LOSS= [8.996955871582031, 0.0] D_LOSS= [0.19662490487098694, 0.5]
EPOCH= 1 BATCH= 7 G_LOSS= [7.058674335479736, 0.0] D_LOSS= [0.2572683095932007, 0.5]
EPOCH= 1 BATCH= 8 G_LOSS= [5.982072830200195, 0.0] D_LOSS= [0.27769196033477783, 0.5]
EPOCH= 1 BATCH= 9 G_LOSS= [7.0629119873046875, 0.0] D_LOSS= [0.23666176199913025, 0.5]
EPOCH= 1 BATCH= 10 G_LOSS= [7.030662536621094, 0.0] D_LOSS= [0.22547750174999237, 0.5]
EPOCH= 1 BATCH= 11 G_LOSS= [8.184053421020508, 0.0] D_LOSS= [0.20659276843070984, 0.5]
EPOCH= 1 BATCH= 12 G_LOSS= [9.613183975219727, 0.0] D_L

EPOCH= 1 BATCH= 96 G_LOSS= [10.518138885498047, 0.0] D_LOSS= [0.1635397970676422, 0.5]
EPOCH= 1 BATCH= 97 G_LOSS= [10.39091968536377, 0.0] D_LOSS= [0.16333939135074615, 0.5]
EPOCH= 1 BATCH= 98 G_LOSS= [10.415704727172852, 0.0] D_LOSS= [0.16322818398475647, 0.5]
EPOCH= 1 BATCH= 99 G_LOSS= [10.566773414611816, 0.0] D_LOSS= [0.16310523450374603, 0.5]
EPOCH= 1 BATCH= 100 G_LOSS= [10.60588264465332, 0.0] D_LOSS= [0.163274347782135, 0.5]
EPOCH= 1 BATCH= 101 G_LOSS= [10.505582809448242, 0.0] D_LOSS= [0.16322200000286102, 0.5]
EPOCH= 1 BATCH= 102 G_LOSS= [10.453165054321289, 0.0] D_LOSS= [0.16316142678260803, 0.5]
EPOCH= 1 BATCH= 103 G_LOSS= [10.597972869873047, 0.0] D_LOSS= [0.16323992609977722, 0.5]
EPOCH= 1 BATCH= 104 G_LOSS= [10.671052932739258, 0.0] D_LOSS= [0.1632726937532425, 0.5]
EPOCH= 1 BATCH= 105 G_LOSS= [10.572541236877441, 0.0] D_LOSS= [0.16314655542373657, 0.5]
EPOCH= 1 BATCH= 106 G_LOSS= [10.526443481445312, 0.0] D_LOSS= [0.1631225049495697, 0.5]
EPOCH= 1 BATCH= 107 G_LOSS= [10.

EPOCH= 1 BATCH= 189 G_LOSS= [11.67892074584961, 0.0] D_LOSS= [0.16280508041381836, 0.5]
EPOCH= 1 BATCH= 190 G_LOSS= [11.738187789916992, 0.0] D_LOSS= [0.16287720203399658, 0.5]
EPOCH= 1 BATCH= 191 G_LOSS= [11.603071212768555, 0.0] D_LOSS= [0.1628602147102356, 0.5]
EPOCH= 1 BATCH= 192 G_LOSS= [11.668560981750488, 0.0] D_LOSS= [0.16281428933143616, 0.5]
EPOCH= 1 BATCH= 193 G_LOSS= [11.735733032226562, 0.0] D_LOSS= [0.16283603012561798, 0.5]
EPOCH= 1 BATCH= 194 G_LOSS= [11.7344970703125, 0.0] D_LOSS= [0.16277453303337097, 0.5]
EPOCH= 1 BATCH= 195 G_LOSS= [11.72941780090332, 0.0] D_LOSS= [0.16274619102478027, 0.5]
EPOCH= 1 BATCH= 196 G_LOSS= [11.753137588500977, 0.0] D_LOSS= [0.16275939345359802, 0.5]
EPOCH= 1 BATCH= 197 G_LOSS= [11.739486694335938, 0.0] D_LOSS= [0.1627899408340454, 0.5]
EPOCH= 1 BATCH= 198 G_LOSS= [11.813899040222168, 0.0] D_LOSS= [0.16279244422912598, 0.5]
EPOCH= 1 BATCH= 199 G_LOSS= [11.728631973266602, 0.0] D_LOSS= [0.16281552612781525, 0.5]
EPOCH= 1 BATCH= 200 G_LOSS=

EPOCH= 1 BATCH= 282 G_LOSS= [11.998025894165039, 0.0] D_LOSS= [0.16274429857730865, 0.5]
EPOCH= 1 BATCH= 283 G_LOSS= [12.181488037109375, 0.0] D_LOSS= [0.16273978352546692, 0.5]
EPOCH= 1 BATCH= 284 G_LOSS= [12.028395652770996, 0.0] D_LOSS= [0.16283246874809265, 0.5]
EPOCH= 1 BATCH= 285 G_LOSS= [12.047834396362305, 0.0] D_LOSS= [0.1627422720193863, 0.5]
EPOCH= 1 BATCH= 286 G_LOSS= [12.093996047973633, 0.0] D_LOSS= [0.16269715130329132, 0.5]
EPOCH= 1 BATCH= 287 G_LOSS= [12.071483612060547, 0.0] D_LOSS= [0.16269448399543762, 0.5]
EPOCH= 1 BATCH= 288 G_LOSS= [12.125840187072754, 0.0] D_LOSS= [0.1627337634563446, 0.5]
EPOCH= 1 BATCH= 289 G_LOSS= [12.040766716003418, 0.0] D_LOSS= [0.16273993253707886, 0.5]
EPOCH= 1 BATCH= 290 G_LOSS= [12.04292106628418, 0.0] D_LOSS= [0.1627175509929657, 0.5]
EPOCH= 1 BATCH= 291 G_LOSS= [12.11871337890625, 0.0] D_LOSS= [0.162693589925766, 0.5]
EPOCH= 1 BATCH= 292 G_LOSS= [12.044931411743164, 0.0] D_LOSS= [0.16272065043449402, 0.5]
EPOCH= 1 BATCH= 293 G_LOSS= 

EPOCH= 1 BATCH= 375 G_LOSS= [12.545180320739746, 0.0] D_LOSS= [0.1626296490430832, 0.5]
EPOCH= 1 BATCH= 376 G_LOSS= [12.447127342224121, 0.0] D_LOSS= [0.16269178688526154, 0.5]
EPOCH= 1 BATCH= 377 G_LOSS= [12.499171257019043, 0.0] D_LOSS= [0.16269779205322266, 0.5]
EPOCH= 1 BATCH= 378 G_LOSS= [12.534160614013672, 0.0] D_LOSS= [0.16264517605304718, 0.5]
EPOCH= 1 BATCH= 379 G_LOSS= [12.509571075439453, 0.0] D_LOSS= [0.162690207362175, 0.5]
EPOCH= 1 BATCH= 380 G_LOSS= [12.463714599609375, 0.0] D_LOSS= [0.1627461016178131, 0.5]
EPOCH= 1 BATCH= 381 G_LOSS= [12.481019020080566, 0.0] D_LOSS= [0.16267022490501404, 0.5]
EPOCH= 1 BATCH= 382 G_LOSS= [12.50102710723877, 0.0] D_LOSS= [0.1626662164926529, 0.5]
EPOCH= 1 BATCH= 383 G_LOSS= [12.455687522888184, 0.0] D_LOSS= [0.16267403960227966, 0.5]
EPOCH= 1 BATCH= 384 G_LOSS= [12.471343994140625, 0.0] D_LOSS= [0.16264647245407104, 0.5]
EPOCH= 1 BATCH= 385 G_LOSS= [12.488780975341797, 0.0] D_LOSS= [0.16264301538467407, 0.5]
EPOCH= 1 BATCH= 386 G_LOSS=

...

EPOCH= 2 BATCH= 94 G_LOSS= [13.382323265075684, 0.0] D_LOSS= [0.1626356840133667, 0.5]
EPOCH= 2 BATCH= 95 G_LOSS= [13.091126441955566, 0.0] D_LOSS= [0.16278958320617676, 0.5]
EPOCH= 2 BATCH= 96 G_LOSS= [13.314403533935547, 0.0] D_LOSS= [0.16288460791110992, 0.5]
EPOCH= 2 BATCH= 97 G_LOSS= [13.348857879638672, 0.0] D_LOSS= [0.16263896226882935, 0.5]
EPOCH= 2 BATCH= 98 G_LOSS= [13.14712905883789, 0.0] D_LOSS= [0.16270962357521057, 0.5]
EPOCH= 2 BATCH= 99 G_LOSS= [13.287952423095703, 0.0] D_LOSS= [0.16273769736289978, 0.5]
EPOCH= 2 BATCH= 100 G_LOSS= [13.392166137695312, 0.0] D_LOSS= [0.16263478994369507, 0.5]
EPOCH= 2 BATCH= 101 G_LOSS= [13.125061988830566, 0.0] D_LOSS= [0.16277164220809937, 0.5]
EPOCH= 2 BATCH= 102 G_LOSS= [13.321918487548828, 0.0] D_LOSS= [0.1628323793411255, 0.5]
EPOCH= 2 BATCH= 103 G_LOSS= [13.367938995361328, 0.0] D_LOSS= [0.16264384984970093, 0.5]
EPOCH= 2 BATCH= 104 G_LOSS= [13.114757537841797, 0.0] D_LOSS= [0.16270554065704346, 0.5]
EPOCH= 2 BATCH= 105 G_LOSS= [1

EPOCH= 2 BATCH= 187 G_LOSS= [13.485713958740234, 0.0] D_LOSS= [0.1626589447259903, 0.5]
EPOCH= 2 BATCH= 188 G_LOSS= [13.466399192810059, 0.0] D_LOSS= [0.16264373064041138, 0.5]
EPOCH= 2 BATCH= 189 G_LOSS= [13.37570571899414, 0.0] D_LOSS= [0.16261908411979675, 0.5]
EPOCH= 2 BATCH= 190 G_LOSS= [13.502791404724121, 0.0] D_LOSS= [0.16265209019184113, 0.5]
EPOCH= 2 BATCH= 191 G_LOSS= [13.466460227966309, 0.0] D_LOSS= [0.1626635491847992, 0.5]
EPOCH= 2 BATCH= 192 G_LOSS= [13.381485939025879, 0.0] D_LOSS= [0.1626269668340683, 0.5]
EPOCH= 2 BATCH= 193 G_LOSS= [13.557951927185059, 0.0] D_LOSS= [0.16267770528793335, 0.5]
EPOCH= 2 BATCH= 194 G_LOSS= [13.46872329711914, 0.0] D_LOSS= [0.16268183290958405, 0.5]
EPOCH= 2 BATCH= 195 G_LOSS= [13.382427215576172, 0.0] D_LOSS= [0.16262458264827728, 0.5]
EPOCH= 2 BATCH= 196 G_LOSS= [13.579137802124023, 0.0] D_LOSS= [0.16269007325172424, 0.5]
EPOCH= 2 BATCH= 197 G_LOSS= [13.43337631225586, 0.0] D_LOSS= [0.16271987557411194, 0.5]
EPOCH= 2 BATCH= 198 G_LOSS=

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x0000025F7E9D3160>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 545, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1262, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


EPOCH= 2 BATCH= 199 G_LOSS= [13.573439598083496, 0.0] D_LOSS= [0.1626489907503128, 0.5]
EPOCH= 2 BATCH= 200 G_LOSS= [13.433910369873047, 0.0] D_LOSS= [0.1627027988433838, 0.5]
EPOCH= 2 BATCH= 201 G_LOSS= [13.460295677185059, 0.0] D_LOSS= [0.1626346856355667, 0.5]
EPOCH= 2 BATCH= 202 G_LOSS= [13.58493423461914, 0.0] D_LOSS= [0.16263523697853088, 0.5]
EPOCH= 2 BATCH= 203 G_LOSS= [13.407271385192871, 0.0] D_LOSS= [0.16267669200897217, 0.5]
EPOCH= 2 BATCH= 204 G_LOSS= [13.44314193725586, 0.0] D_LOSS= [0.16264277696609497, 0.5]
EPOCH= 2 BATCH= 205 G_LOSS= [13.503600120544434, 0.0] D_LOSS= [0.16261540353298187, 0.5]
EPOCH= 2 BATCH= 206 G_LOSS= [13.325804710388184, 0.0] D_LOSS= [0.16267254948616028, 0.5]
EPOCH= 2 BATCH= 207 G_LOSS= [13.42171859741211, 0.0] D_LOSS= [0.16269823908805847, 0.5]
EPOCH= 2 BATCH= 208 G_LOSS= [13.426803588867188, 0.0] D_LOSS= [0.1626233160495758, 0.5]
EPOCH= 2 BATCH= 209 G_LOSS= [13.215561866760254, 0.0] D_LOSS= [0.1626560389995575, 0.5]
EPOCH= 2 BATCH= 210 G_LOSS= [

KeyboardInterrupt: 

In [ ]:
#EPOCH= 1 BATCH= 1 G_LOSS= [6.25101900100708, 0.0] D_LOSS= [0.6248681545257568, 0.5]
#EPOCH= 1 BATCH= 2 G_LOSS= [3.66963791847229, 0.0] D_LOSS= [0.41628673672676086, 0.5]
#EPOCH= 1 BATCH= 3 G_LOSS= [4.668774127960205, 0.0] D_LOSS= [0.20419344305992126, 0.5]

In [ ]:
#위 방법 보다 실제로 쓰기 쉬운 코드 --> DCGAN